### Jhanelle Williams
###Assignment 7

In [277]:
#version of software used
pd.__version__

'0.17.0'

In [278]:
#version of software used
np.__version__

'1.10.1'

In [279]:
# import libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#stats library

import statsmodels.api as sm
import scipy

#T-test is imported to complete the statistical analysis

from scipy.stats import ttest_ind

from scipy import stats

#The function below is used to show the plots within the notebook

%matplotlib inline

In [160]:
# Load data 

decays_data=pd.read_csv('/Users/Jhanelle/Desktop/behav_new2.dat', delimiter=' ')

In [91]:
# show data

decays_data

# Show only the first 5 rows of the data

decays_data.head(n=20)

,date,plate,time,strain,frame,ID,persistance,area,speed,angularspeed,...,bias,pathlen,curve,dir,loc_x,loc_y,vel_x,vel_y,orient,crab
0,20141118,20141118_131037,15.028,wild-type,0,1,234.9,0.096228,NaN,NaN,...,1,0.000,38.2,0,29.0036,29.4805,NaN,NaN,103.3,NaN
1,20141118,20141118_131037,15.069,wild-type,1,1,234.9,0.096957,NaN,NaN,...,1,0.010,32.7,0,29.0076,29.4713,NaN,NaN,103.1,NaN
2,20141118,20141118_131037,15.109,wild-type,2,1,234.9,0.096957,NaN,NaN,...,1,0.019,34.0,0,29.0113,29.4630,NaN,NaN,102.8,NaN
3,20141118,20141118_131037,15.148,wild-type,3,1,234.9,0.095499,NaN,NaN,...,1,0.027,33.5,0,29.0138,29.4550,NaN,NaN,102.2,NaN
4,20141118,20141118_131037,15.186,wild-type,4,1,234.9,0.095499,NaN,NaN,...,1,0.036,34.2,0,29.0172,29.4475,NaN,NaN,102.1,NaN
5,20141118,20141118_131037,15.229,wild-type,5,1,234.9,0.096228,NaN,NaN,...,1,0.044,34.4,0,29.0197,29.4397,NaN,NaN,102.3,NaN
6,20141118,20141118_131037,15.270,wild-type,6,1,234.9,0.100602,NaN,NaN,...,1,0.054,35.0,0,29.0225,29.4302,NaN,NaN,102.4,NaN
7,20141118,20141118_131037,15.310,wild-type,7,1,234.9,0.100602,0.1914,1.3,...,1,0.061,33.8,0,29.0251,29.4233,0.0640,-0.1804,102.3,0.0240
8,20141118,20141118_131037,15.350,wild-type,8,1,234.9,0.099873,0.1844,0.9,...,1,0.069,33.7,0,29.0278,29.4162,0.0593,-0.1746,102.4,0.0205
9,20141118,20141118_131037,15.387,wild-type,9,1,234.9,0.099144,0.1807,1.1,...,1,0.077,33.7,0,29.0309,29.4083,0.0579,-0.1712,102.3,0.0201


### Question and Hypothesis for research:

What is the difference in speed between the wild type and the mutant_1 worms?

Hypothesis: Mutant_1 worms are quicker at sensing carbon dioxide in the study in comparison to the wild type worms which are not resulting in mutant_1 worms having a greater speed. 


###Statistical analysis of the data

To test the hypothesis a two sampled t-test wll be applied to the required data frame. 

### General Code used to calculate the two sampled t-test

-> rvs1=stats.norm.rvs(loc=mean1, scale=variance, size= n)

-> rvs2=stats.norm.rvs(loc=mean2, scale=variance2, size= n)

--> stats.ttest_ind(rvs1,rvs2, equal_var=False)

In [280]:
# Here we extract only two columns from the data as these are the main variables for the statistcal analysis

strain_df=decays_data[['strain','speed']]
strain_df.head()

,strain,speed
0,wild-type,NaN
1,wild-type,NaN
2,wild-type,NaN
3,wild-type,NaN
4,wild-type,NaN


In [56]:
# Eliminate NaN from the dataset 

strain_df=strain_df.dropna()
strain_df.head()

,strain,speed
7,wild-type,0.1914
8,wild-type,0.1844
9,wild-type,0.1807
10,wild-type,0.1746
11,wild-type,0.1741


In [200]:
#Resample the data to group by strain

strain_resampled=strain_df.groupby('strain')
strain_resampled.head()


,strain,speed
7,wild-type,0.1914
8,wild-type,0.1844
9,wild-type,0.1807
10,wild-type,0.1746
11,wild-type,0.1741
114314,mutant_1,0.1257
114315,mutant_1,0.1204
114316,mutant_1,0.1184
114317,mutant_1,0.1213
114318,mutant_1,0.1271


In [281]:
# Extract mutant_1 strain from the resampled data as this allows for easier comparison of the groups for the t test analysis

mutant_strain=strain_df.groupby('strain').get_group('mutant_1')
mutant_strain.head()

,strain,speed
114307,mutant_1,NaN
114308,mutant_1,NaN
114309,mutant_1,NaN
114310,mutant_1,NaN
114311,mutant_1,NaN


In [285]:
# Calculate the sample size for the mutant_1 strain

len(mutant_strain)

265737

In [284]:
# Extract wild-type strain from the resampled data as this allows for easier comparison of the groups for the t test analysis

wild_type=strain_df.groupby('strain').get_group('wild-type')
wild_type.head()

,strain,speed
0,wild-type,NaN
1,wild-type,NaN
2,wild-type,NaN
3,wild-type,NaN
4,wild-type,NaN


In [286]:
# Calculate the sample size (n) for the wild-type strain


len(wild_type)

241417

In [287]:
# Find the mean speed for the categorical data to confirm the difference in the mean as this helps to calculate the t-test

strain_resampled.mean()

,speed
strain,
mutant_1,0.093874
wild-type,0.109037


In [288]:
# Variance function defined using a loop 

def variance(name):
    var=(name.std())**1/2
    return (var)


In [289]:
#Print variance for wild_type

variance(wild_type)

speed    0.03242
dtype: float64

In [290]:
# #Print variance for mutant_strain

variance(mutant_strain)

speed    0.029144
dtype: float64

In [291]:
#Define array and substitute values as calculated above for this array

rvs1=stats.norm.rvs(loc=0.093872, scale=0.03242, size= 239963)
rvs1

array([ 0.08713935,  0.04282218,  0.10882944, ...,  0.14650845,
        0.06602893,  0.11761714])

In [292]:
#Define array and substitute values as calculated above for this array

rvs2=stats.norm.rvs(loc=0.109037, scale=0.029144, size= 264600)
rvs2

array([ 0.13038803,  0.12978784,  0.10017476, ...,  0.07721623,
        0.09940389,  0.12177672])

In [276]:
# Here we calculate our t-test, where the first value is the t score and p value is 0 which shows that it is statistically significant
stats.ttest_ind(rvs1,rvs2, equal_var=False)


(-174.38132493252263, 0.0)

### Discussion of Statistical Data

Results:
T value= -174.38
p value= 0

The two-sampled t-test shows significant evidence that leds to the rejection of the null hypothesis. The p value is 0 which is less than the signficance level (0.05), this knowledge proves that the hypothesis is rejected. Therefore, there is a difference between the speed of mutant_1 and wild -type strains.  


